# Задание 2 "Мониторинг в сельском хозяйстве"

---

**Курс:** Проектирование систем машинного обучения  
**Студент:** Безмылов А.П.    
**Группа:** М80-210СВ-24  


---

# 1. Введение и постановка задачи
---
Задача: Спроектируйте систему с использованием изображений с дронов для выявления болезней сельскохозяйственных культур. Сосредоточьтесь на обработке больших объемов изображений и обновлении моделей.

В сельском хозяйстве заболевания растений являются одной из ключевых причин снижения урожайности и роста операционных затрат. Несвоевременное выявление фитопатологий приводит к распространению болезней на большие площади и необходимости массового применения средств защиты растений.

В рамках данного проекта разрабатывается **промышленная ML-система мониторинга здоровья сельскохозяйственных культур**, основанная на анализе изображений, получаемых с дронов, оснащённых мультиспектральными камерами. Система ориентирована на автоматическую обработку больших потоков изображений, выявление и локализацию зон поражения растений и предоставление аналитики агрономам вблизи реального времени.

Проектируемая система должна быть масштабируемой, отказоустойчивой и способной автоматически адаптироваться к изменениям данных (сезонность, новые заболевания, изменения условий съёмки).



## Бизнес-цели

1. **Снижение потерь урожая**  
   Раннее выявление заболеваний и стрессовых зон растений с целью снижения потерь урожайности на **30–40%**.

2. **Оптимизация затрат на агроскаутинг**  
   Сокращение ручных осмотров полей и затрат на персонал до **60%** за счёт автоматизированного анализа данных с дронов.

3. **Повышение эффективности обработки полей**  
   Обеспечение точечного внесения удобрений и средств защиты растений, что позволяет снизить перерасход химикатов до **25%**.


## Требования к системе

### Производительность и нагрузка

- **Задержка (latency):**  
  Время обработки одного изображения — не более **490 мс**.

- **Пропускная способность (throughput):**  
  Поддержка пиковых нагрузок до **2 334 запросов в секунду (RPS)**, возникающих при массовых облётах полей.

- **Пользовательская нагрузка:**  
  До **4 673 977 активных пользователей в день (DAU)**, включая агрономов, фермеров и автоматические сервисы анализа.

### Надёжность и масштабируемость

- Горизонтальное масштабирование inference-сервисов.  
- Отсутствие single point of failure.  
- Поддержка multi-AZ / multi-ЦОД размещения ключевых компонентов.
---
# Часть 1. Формулировка ML-задачи и выбор модели
---
# 1.1 Определение ML-задачи

### Формулировка задачи

Задача мониторинга состояния сельскохозяйственных культур формулируется как задача **детекции и сегментации заболеваний растений на изображениях с дронов**.

Для каждого входного изображения необходимо:

- определить наличие заболеваний;
- классифицировать тип заболевания;
- локализовать поражённые участки на изображении.

### Обоснование постановки задачи

- Одно изображение может содержать несколько зон поражения с разной степенью выраженности.  
- Для принятия агрономических решений важно понимать **пространственное распределение** заболеваний по полю.  
- Использование только классификации без локализации не позволяет оценить площадь поражения и спланировать обработку.

Таким образом, задача относится к классу **object detection / instance segmentation**.

### Входные данные

- Изображения с дронов:
  - RGB (опционно);
  - мультиспектральные каналы (NIR, Red Edge, Red).
- Метаданные съёмки:
  - геопозиция;
  - время съёмки;
  - параметры полёта.

### Выходные данные

- Bounding boxes или маски поражённых участков.
- Класс заболевания.
- Оценка степени поражения (confidence score).

### Целевая переменная

- Координаты bounding boxes (или сегментационные маски).  
- Класс заболевания для каждой области.

# 1.2 Выбор модели

Для решения поставленной задачи были рассмотрены несколько архитектур компьютерного зрения.

### Модель 1: YOLOv11

**Преимущества:**

- высокая скорость инференса, критичная при нагрузке до 2 334 RPS;
- одновременная детекция и классификация;
- хорошая масштабируемость при запуске в Kubernetes;
- поддержка экспорта в ONNX и оптимизации под GPU;
- эффективная работа с мультиспектральными каналами.

**Недостатки:**

- меньшая точность сегментации по сравнению с полноразмерными segmentation-моделями;  
- зависимость качества от точности разметки.

### Модель 2: U-Net / DeepLabv3+

**Преимущества:**

- высокая точность пиксельной сегментации;  
- хорошая интерпретируемость результатов.

**Недостатки:**

- высокая вычислительная сложность;  
- низкая скорость инференса;  
- высокая стоимость разметки данных.

### Модель 3: EfficientDet

**Преимущества:**

- высокая точность детекции;  
- поддержка объектов разного масштаба.

**Недостатки:**

- более сложная архитектура;  
- более низкая скорость инференса по сравнению с YOLO.

## Выбор модели для системы

В рамках проекта выбрана модель **YOLOv11m**.

### Обоснование выбора

- соответствие требованиям по latency (**< 490 мс**);  
- возможность обработки пиковых нагрузок (**2 334 RPS**) при горизонтальном масштабировании;  
- баланс между скоростью и качеством;  
- удобство деплоя в Kubernetes и интеграции с GPU-кластером;  
- поддержка дообучения и адаптации под новые типы заболеваний.
---
# Часть 2: Проектирование архитектуры
---
# 1. Высокоуровневая архитектура системы

Высокоуровневая архитектура системы (High-Level Design, HLD) описывает промышленную ML-платформу для мониторинга состояния сельскохозяйственных культур на основе мультиспектральных данных, получаемых с дронов. Система охватывает полный жизненный цикл данных и моделей: от сбора изображений и подготовки датасетов до **real-time инференса**, мониторинга качества и автоматического переобучения моделей.

Архитектура спроектирована с учётом следующих целевых требований:

- **Latency P95:** < 490 мс для онлайн-инференса  
- **Пиковая нагрузка:** до 2 334 RPS  
- **Доступность:** не ниже 99.95%  
- **Масштабируемость:** горизонтальная, отсутствие единой точки отказа  

Система логически разделена на три взаимосвязанных блока:

1. **Offline Training** — обучение и обновление модели  
2. **Real-time Inference** — онлайн-анализ данных  
3. **Monitoring & Feedback Loop** — мониторинг и контроль деградации  

## 1.1 Offline Training (Обучение модели)

Блок **Offline Training** отвечает за подготовку данных, обучение моделей и их безопасную передачу в production.

**Основные компоненты:**

- **Drones with Multispectral Cameras (NIR / Red Edge / Red, RGB опционально)**  
  Выполняют съёмку сельскохозяйственных угодий. Мультиспектральные каналы позволяют выявлять стресс растений на ранних стадиях, когда изменения в RGB-диапазоне ещё не выражены.

- **API Gateway (NGINX / L7 + OAuth)**  
  Приём данных, первичная валидация запросов, аутентификация и балансировка нагрузки между зонами доступности.

- **Kafka Image Stream**  
  Буферизация потоков изображений и метаданных при пиковой нагрузке (**до 2 334 RPS**, ~1.16 ГБ/с), предотвращает потерю данных и сглаживает пиковые нагрузки после массовых облётов.

- **Object Storage (S3 / GeoTIFF)**  
  Хранение сырых мультиспектральных изображений и производных спектральных карт. Данные делятся на горячее и холодное хранилище в зависимости от актуальности.

- **Spectral Feature Extraction**  
  Вычисление агроспецифических признаков (NDVI, NDRE, GNDVI и др.), формирующих входное представление для модели и повышающих устойчивость к визуальному шуму.

- **Automated Labeling & Pseudo-Labels**  
  Генерация псевдо-меток на основе предыдущих версий моделей и эвристических правил. Финальная валидация выполняется агрономами для формирования ground truth.

- **Apache Airflow**  
  Оркестрация пайплайнов подготовки данных и обучения, включая обработку дрифта и плановое переобучение.

- **Training Pipeline (GPU Compute Cluster — NVIDIA A100)**  
  Обучение модели сегментации (YOLOv11m) на GPU-кластере A100, что обеспечивает высокую пропускную способность и поддержку сложных аугментаций.

- **MLflow Model Registry**  
  Хранение версий моделей, параметров обучения, метрик качества и референсных распределений для последующего мониторинга дрифта.

- **Deploy to Inference**  
  Только проверенные модели, прошедшие валидацию, передаются в онлайн-инференс.

## 1.2 Real-time Inference (Онлайн-анализ)

Блок **Real-time Inference** обеспечивает оперативную обработку данных и формирование результатов для агрономов в near-real-time режиме.

**Основные компоненты:**

- **Incoming Drone Images**  
  Поток изображений поступает после облёта или в режиме, близком к реальному времени.

- **Kubernetes Cluster + GPU Operator**  
  Инференс развёрнут в Kubernetes-кластере с GPU Operator. Используется **13 GPU NVIDIA T4** с автоскейлингом **4 → 15 инстансов**, что обеспечивает требуемую пропускную способность и **latency < 490 мс**.

- **Inference Service (YOLOv11m + Multispectral Features)**  
  Сегментация зон поражения и оценка степени стресса растений на основе мультиспектральных данных и спектральных индексов.

- **Output Storage**  
  - **PostgreSQL / Greenplum** — долговременное хранение результатов анализа и истории заболеваний по полям  
  - **Redis** — кэш последних и часто запрашиваемых результатов для снижения нагрузки и ускорения отклика

## 1.3 Monitoring & Feedback Loop

Блок обеспечивает контроль качества работы системы, обнаружение деградации модели и замыкание цикла обучения.

**Основные компоненты:**

- **Kafka Feedback Stream**  
  Асинхронная очередь для фидбека от агрономов и событий эксплуатации, используется для запуска повторного обучения.

- **Monitoring (Prometheus + Grafana)**  
  Сбор и визуализация:  
  - инфраструктурных метрик (CPU/GPU, latency, throughput)  
  - метрик модели (mAP, Precision, Recall)  
  - метрик дрифта данных и предсказаний (PSI, KS)

- **Feedback Loop**  
  При обнаружении деградации модели или превышении порогов дрифта автоматически инициируется дообучение в Offline Training контуре.

- **Agronomic Dashboard**  
  Пользовательский интерфейс для агрономов, отображающий состояние полей, динамику заболеваний и рекомендации по обработке.

**Схема Высокоуровневой архитектуры:**

![Высокоуровневая архитектура системы](higharch.png)

# 2. Архитектура Data Pipeline

Data Pipeline описывает процесс сбора, валидации, разметки и подготовки мультиспектральных данных, используемых для обучения и инференса модели мониторинга состояния сельскохозяйственных культур.  
Пайплайн сочетает **оффлайн batch-обработку** (Python / batch-скрипты для нормализации, коррекции и расчёта спектральных индексов) и **онлайн-обновление feature store** (Redis для хранения актуальных агро-признаков и метаданных), что позволяет одновременно обеспечивать воспроизводимость обучения и низкую задержку при инференсе.

## 2.1 Сбор данных (Data Ingestion)

На первом этапе осуществляется приём исходных данных с дронов и их первичная буферизация.

- **Drones with Multispectral Cameras**  
  Беспилотные летательные аппараты выполняют съёмку сельскохозяйственных полей в мультиспектральных диапазонах (NIR, Red Edge, Red); RGB-канал используется опционально для визуальной проверки и экспертного анализа.

- **API Gateway**  
  Обеспечивает приём изображений и метаданных с дронов, аутентификацию, контроль формата запросов и балансировку нагрузки между зонами доступности.

- **Kafka Image Stream**  
  Используется для асинхронной буферизации потока изображений и метаданных, что позволяет сглаживать пиковые нагрузки после массовых облётов и предотвращать потерю данных.

## 2.2 Архивирование и хранение сырых данных

После приёма данные сохраняются для последующей оффлайн-обработки и переобучения моделей.

- **Object Storage (S3-совместимое хранилище)**  
  Сырые мультиспектральные изображения и связанные метаданные архивируются в объектном хранилище.  
  Данные хранятся с разделением на горячее и холодное хранилище в зависимости от давности и частоты использования.

## 2.3 Оффлайн обработка и подготовка датасетов (Offline Batch Processing)

Оффлайн-пайплайн отвечает за формирование обучающих и валидационных наборов данных.

- **Data Validation**  
  Проверка корректности формата изображений, целостности спектральных каналов и метаданных (GPS-координаты, параметры сенсора, время облёта).

- **Data Preprocessing / Enrichment**
  - нормализация и приведение изображений к целевому формату;
  - коррекция освещения и сенсорных искажений;
  - геометрическая коррекция и выравнивание снимков;
  - обогащение данных контекстными признаками (погодные условия, тип почвы, агрономические характеристики).

- **Spectral Feature Extraction**  
  Вычисление агроспецифических спектральных индексов (NDVI, NDRE, GNDVI и др.).

- **Pseudo-labeling**  
  Генерация псевдо-меток на основе предыдущих версий моделей для pre-training и data augmentation.

- **Human Labeling UI (Roboflow)**  
  Агрономы и эксперты проверяют и корректируют разметку ключевых участков, формируя ground truth.

- **Dataset Management (DVC)**
  - **Train Dataset** — обучающая выборка;
  - **Eval Dataset** — validation и test выборки для оценки модели.

## 2.4 Онлайн Feature Store и инференс  

- **Redis Feature Store**  
  Хранение последних агро-признаков и метаданных по каждому полю с задержкой доступа менее 10 мс.

- **Inference API**  
  Принимает новые мультиспектральные снимки после облёта.

- **Context Merger**  
  Асинхронно обновляет Feature Store и агрегирует контекст (последние N снимков по полю).

- **YOLOv11m (fine-tuned)**  
  Выполняет сегментацию зон поражения и оценку степени стресса растений.

- **Output Storage**
  - **Greenplum / SQL** — долговременное хранение результатов анализа и истории состояния полей;
  - **Redis Cache** — быстрый доступ к последним результатам.

## Выводы

- Обработка данных в **near real-time** за счёт Kafka и Redis.
- Автоматический мониторинг качества данных и **drift detection** с триггером на переобучение в MLflow.
- Снижение объёма ручной разметки через **pseudo-labels** и human-in-the-loop.
- Полная воспроизводимость экспериментов и версионирование данных и моделей.
- Высокая масштабируемость и низкая задержка (**latency ≤ 385 мс**) при пиковой нагрузке до **2 334 RPS**.


**Схема Архитектуры Data Pipeline:**

![Архитектура Data Pipeline](high_level_architecture.svg)

# 3. Архитектура Training Pipeline

Training Pipeline описывает автоматизированный и воспроизводимый процесс обучения и дообучения модели детекции поражённых зон сельскохозяйственных культур на основе мультиспектральных данных.  
Пайплайн реализован с использованием MLOps-инструментов (**Kubeflow Orchestrator, MLflow, DVC**) и GPU-кластера и поддерживает обучение, валидацию и безопасное развертывание моделей в промышленной среде.

Pipeline может запускаться по расписанию, при накоплении новых размеченных данных или при обнаружении **data / concept drift** в production-потоке.

## 3.1 Триггеры запуска обучения (Triggers)

Обучение модели инициируется одним из следующих событий:

- **Scheduled Trigger**  
  Автоматический запуск по расписанию (ежедневно или еженедельно) для регулярного обновления модели.

- **New Labeled Data Trigger**  
  Запуск при накоплении заданного количества новых размеченных изображений, подготовленных в Data Pipeline и версионируемых с помощью DVC.

- **Data / Concept Drift Trigger**  
  Production-данные из Kafka, Redis и S3 анализируются на наличие дрифта.  
  При превышении порогов (PSI > 0.2, KS > 0.1 или снижение Precision / Recall более чем на 10%) Kubeflow Orchestrator автоматически инициирует retraining.

## 3.2 Подготовка данных к обучению

- Финальный датасет загружается из **Data Pipeline (DVC)**.
- Данные разделяются на **Train / Validation / Test** выборки с учётом временной структуры (time-based split).
- Выполняется балансировка классов с учётом сезонности и географии.
- Применяются аугментации:
  - геометрические трансформации;
  - изменение яркости и добавление шума;
  - спектральные трансформации.
- В датасет добавляются **pseudo-labels** для расширения обучающей выборки и pre-training.

## 3.3 Обучение и fine-tuning модели

- В качестве базовой архитектуры используется **YOLOv11m**, инициализированная предобученными весами (COCO).
- Fine-tuning выполняется на мультиспектральных данных с повышенным приоритетом для последних и high-drift примеров.
- Подбор гиперпараметров осуществляется с использованием **Optuna** (learning rate, batch size, политики аугментаций).
- Обучение выполняется на распределённом **GPU-кластере NVIDIA A100** с использованием mixed precision для ускорения вычислений.
- Все параметры и промежуточные результаты логируются в **MLflow**.

## 3.4 Валидация и контроль качества модели

- Оценка модели проводится на **Validation и Test выборках**, включая временные окна с зафиксированным дрифтом.
- Используемые метрики:
  - **mAP@0.5**
  - **Precision**
  - **Recall**
  - **F1-score**
- Дополнительно выполняется анализ bias:
  - subgroup F1 по полям;
  - по культурам;
  - по регионам.
- Все метрики сохраняются в **MLflow Metrics** для сравнения версий моделей.
- При недостижении целевых значений pipeline автоматически возвращается к этапу обучения.
- При успешной валидации запускается **Shadow Testing** (A/B-тестирование на ограниченной группе полей).

## 3.5 Регистрация и развертывание модели

- Успешная модель упаковывается в **Docker-образ**.
- Модель регистрируется в **MLflow Model Registry** с версионированием и сохранением reference distributions для мониторинга дрифта.
- Развертывание выполняется в **Kubernetes** с использованием canary или rolling update без простоя сервиса.

## 3.6 Выводы

- Полностью автоматизированный и воспроизводимый процесс обучения.
- Быстрая адаптация модели к изменению данных за счёт drift-based retraining.
- Безопасный деплой только валидированных моделей.
- Поддержка масштабируемости, отказоустойчивости и мониторинга в реальном времени.

**Схема Архитектуры Training Pipeline:**

![Архитектура Training Pipeline](high_level_architecture.svg)

## 4. Архитектура Inference Pipeline (Serving)

Inference Pipeline описывает систему обработки мультиспектральных данных в реальном времени и предназначен для выявления стрессовых и поражённых зон сельскохозяйственных культур с минимальной задержкой ответа (**< 385 мс**) и поддержкой пиковой нагрузки до **~2 334 RPS**.  
Архитектура ориентирована на агросценарии и учитывает работу со спектральными признаками (NDVI, NDRE, GNDVI), историей поля и масштабируемостью GPU-инференса.

### 4.1 Ввод и приём данных

- **Drones with Multispectral Cameras (NIR, Red Edge, Red; optional RGB)**  
  Формируют мультиспектральные снимки полей после облёта.

- **API Gateway / Load Balancer (NGINX)**  
  Выполняет аутентификацию, rate limiting и распределение запросов между сервисами.

- **Kafka Image Stream**  
  Буферизует поток изображений, обеспечивая асинхронную обработку и устойчивость к пиковым нагрузкам.

*Комментарий:* входной контур отделён от инференса, что предотвращает потерю данных и деградацию latency при массовых облётах.

### 4.2 Контекст и Feature Store

- **Redis Feature Store**  
  Хранит последние агро-признаки и контекст по каждому полю (история замеров, сезонность, метаданные).

- Обновление признаков осуществляется асинхронно через Kafka.

*Комментарий:* использование Redis позволяет учитывать исторический контекст и снижает время подготовки данных для инференса до <10 мс.

### 4.3 Сервис инференса

- **Inference Pods (YOLOv11m + Spectral Features)**  
  Выполняют сегментацию поражённых зон и оценку стресса растений на основе мультиспектральных признаков.

- **GPU-ускоренный инференс (NVIDIA T4)**  
  Используется для обработки изображений в реальном времени с поддержкой mixed precision.

- **Kubernetes HPA (Horizontal Pod Autoscaler)**  
  Масштабирует количество подов в зависимости от загрузки GPU, CPU и показателей latency.

*Комментарий:* независимое масштабирование инференса позволяет выдерживать пиковые нагрузки без увеличения времени ответа.

### 4.4 Хранение и визуализация результатов

- **Greenplum / PostgreSQL**  
  Хранение сегментаций, оценок стресса, временных рядов и метаданных по полям.

- **Redis Cache**  
  Кэширование последних результатов для быстрого доступа.

- **Agronomist Dashboard**  
  Визуализация зон стресса и рекомендаций для агрономов, поддержка принятия решений.

*Комментарий:* аналитический слой изолирован от инференса, что исключает влияние пользовательских запросов на производительность модели.

### 4.5 Мониторинг, качество и дрифт

- **Prometheus**  
  Сбор метрик инференса (latency, throughput, GPU utilization).

- **Grafana**  
  Дашборды и алерты для оперативного мониторинга системы.

- **Data / Concept Drift Detection (PSI, KS, MMD)**  
  Анализ изменений распределений признаков и выходов модели.

- При превышении порогов дрифта автоматически формируется триггер на retraining в Training Pipeline.

*Комментарий:* мониторинг и дрифт-детекция обеспечивают стабильное качество модели в условиях сезонных и региональных изменений.

### 4.6 Выводы

- Реальное время обработки мультиспектральных изображений.
- Низкая задержка ответа (**< 385 мс**).
- GPU-ускоренный инференс с автоматическим масштабированием.
- Устойчивость к пиковым нагрузкам и отказам.
- Прямая интеграция с Training Pipeline через дрифт-триггеры.


**Схема Архитектуры Inference Pipeline:**

![Архитектура Inference Pipeline](high_level_architecture.svg)


---

# Часть 3: Расчёты и нефункциональные требования

---

В данном разделе представлены расчёты производительности, требований к хранилищу и масштабируемости промышленной ML-системы для мониторинга состояния сельскохозяйственных культур с использованием мультиспектральных данных с дронов. Расчёты учитывают архитектуру системы, SLA и выбранное оборудование (GPU NVIDIA A100 для обучения и T4 для инференса).

## Нефункциональные требования

Система должна удовлетворять следующим требованиям:

- **Задержка (Latency, P95):** ≤ 490 мс для онлайн-инференса  
- **Нагрузка:**  
  - Активные пользователи: 4 673 977 DAU  
  - Пиковая нагрузка: 2 334 RPS  
- **Надёжность:**  
  - Доступность сервиса: ≥ 99.95%  
  - Отсутствие единой точки отказа (No SPOF)  
- **Масштабируемость:**  
  - Горизонтальное масштабирование инференса через Kubernetes HPA  
  - Автоматическое масштабирование GPU-ресурсов  
- **Производственная готовность:**  
  - Stateless inference  
  - Автоматическое обнаружение деградации модели  
  - Возможность безопасного переобучения  

## 1 Расчёт требований к хранилищу S3

**Исходные данные:**

- Основные мультиспектральные каналы: NIR, Red, Red Edge, float32, 2048×2048  
- Опциональные RGB-каналы: 1024×1024, 8 бит/канал  

**Размер одного изображения:**

- Мультиспектральные: 3 × 2048 × 2048 × 4 байта ≈ **50,3 МБ**  
- RGB (опционально): 3 × 1024 × 1024 × 1 байт ≈ **3 МБ**  
- **Итого:** 50,3 + 3 ≈ **53,3 МБ**  

**Поток данных в секунду:**

- 5000 дронов × 1 снимок/мин → 5000 / 60 ≈ **85 RPS**  
- Поток мультиспектральных каналов: 85 × 50,3 ≈ **4,28 ГБ/с**  
- С RGB: 85 × 53,3 ≈ **4,53 ГБ/с**  

**Объём данных за 8 часов работы дронов:**

- Мультиспектральные: 4,28 × 3600 × 8 ≈ **123,2 ТБ**  
- С RGB: 4,53 × 3600 × 8 ≈ **130,5 ТБ**  

**Итоговое хранилище на 90 дней:**

- Мультиспектральные: 123,2 × 90 ≈ **11,1 ПБ**  
- С RGB: 130,5 × 90 ≈ **11,7 ПБ**  

*Комментарий:* расчет включает только сырые изображения. Маски и артефакты для обучения увеличивают объём на 5–10%.


## Расчёт требований к кэшу Redis

**Параметры:**

- Система обслуживает 4 673 977 DAU  
- Среднее количество запросов на пользователя: 3  
- Уникальные данные: 95%  

**Размер сегментационной маски:**

- 2048 × 2048 / 2^20 ≈ 4 МБ  

**Суточная нагрузка на кэш:**

- 4 673 977 × 3 × 0.95 × 4 МБ ≈ **53 317 ГБ (~52 ТБ)**  

**Шардирование Redis:**

- Один шард ≈ 256 ГБ  
- Необходимо 52 000 / 256 ≈ **205 шардов**  

*Комментарий:* Использование шардирования позволяет держать горячие данные в памяти без перегрузки одного узла.


## 2 Расчёт требований к пропускной способности и инференсу

**Выбранная модель:** YOLOv11m-Seg (мультиспектральные каналы + optional RGB)

**Аппаратное обеспечение для инференса:** NVIDIA T4  

**Производительность одного GPU:**

- Среднее время инференса: ~4,7 мс  
- Пропускная способность: ~212 RPS / GPU  

**Требуемое количество GPU для 2 334 RPS:**

- 2334 / 212 ≈ 11 GPU  
- С учётом отказоустойчивости и нагрузки (N+2): **13 GPU T4**  

**Пиковая нагрузка в сети:**

- Входной трафик: 2 334 × 53,3 МБ ≈ **124 342 МБ/с (~121 ГБ/с)**  
- Выходной трафик: 2 334 × (маска 4 МБ + рамки ~0,1 МБ) ≈ **9,6 ГБ/с**  

*Комментарий:* входной трафик соответствует мультиспектральным каналам с опциональным RGB.


## 3 Масштабируемость и надежность

**Inference Pipeline:**

- Kubernetes + HPA для подов с YOLOv11m-Seg  
- Горизонтальное масштабирование: до **13 GPU T4**, автоскейлинг 4 → 15 при RPS/pending load  
- Stateless pods для упрощения масштабирования  
- Load Balancer + health checks  

**Data Pipeline:**

- Apache Kafka для пиковых нагрузок, горизонтальное масштабирование брокеров и партиций  
- S3-совместимое хранилище для сырых и обработанных изображений  
- Spark / PySpark для пакетной обработки и расчёта индексов  

**Training Pipeline:**

- GPU-кластер NVIDIA A100 для обучения и fine-tuning YOLOv11m  
- Параллельные эксперименты с Optuna для поиска гиперпараметров  
- Возможность добавления GPU-нод для увеличения пропускной способности  

**Feature Store:**

- Версионирование данных и моделей (MLflow + DVC)  
- Горизонтальное масштабирование через шардинг и репликацию  

**API и кэш:**

- API Gateway с горизонтальной балансировкой нагрузки  
- Redis-шардирование для горячих данных и сегментационных масок  

**ЦОД и отказоустойчивость:**

- Размещение в нескольких зонах доступности (Multi-AZ)  
- Statless inference pods и Load Balancer  
- Регулярные бэкапы S3, MLflow и PostgreSQL  
- Канареечное развертывание новых моделей  
- Мониторинг через Grafana / Prometheus и автоматический триггер переобучения при drift  

## SLA и мониторинг

- **Доступность:** 99,95%  
- **Latency P95:** < 490 мс  
- **Throughput:** 2 334 RPS с запасом 20%  
- **Мониторинг:**
  - CPU/GPU/RAM, Network  
  - Data Drift (PSI, KS)  
  - Prediction Drift  
  - Метрики модели: mAP, Precision, Recall  

---

## Список использованных источников

1. **YOLO Documentation**. Ultralytics. URL: [https://docs.ultralytics.com/modes/](https://docs.ultralytics.com/modes/)  
2. **Huyen, C.** Designing Machine Learning Systems. O'Reilly Media, 2022.  
3. **Kleppe, A.** ML System Design: Use Cases and Best Practices. O'Reilly Media, 2020.  
4. **Apache Kafka Documentation**. Apache Software Foundation. URL: [https://kafka.apache.org/documentation/](https://kafka.apache.org/documentation/)  
5. **MLflow Documentation**. Databricks. URL: [https://mlflow.org/docs/latest/index.html](https://mlflow.org/docs/latest/index.html)  
6. **Kubernetes Documentation**. Kubernetes. URL: [https://kubernetes.io/docs/](https://kubernetes.io/docs/)  
7. **Клеппман, М.** Designing Data-Intensive Applications: The Big Ideas Behind Reliable, Scalable, and Maintainable Systems. O’Reilly Media, 2017.  
8. **Аминиан, А., Сюй, А.** System Design. Машинное обучение. Подготовка к сложному интервью. Санкт-Петербург: Питер, 2024.  
9. **MLOps (Machine Learning Operations)**. Методология машинного обучения // TAdviser. URL: [Полный URL отсутствует]  
10. **Amazon Compute Service Level Agreement**. Amazon Web Services. URL: [https://aws.amazon.com/compute/sla/](https://aws.amazon.com/compute/sla/)  